In [1]:
import pandas as pd
import os
import numpy as np


pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = None

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [2]:
from soundbay.utils.metadata_processing import load_n_adapt_raven_annotation_table_to_dv_dataset_requirements

# params

In [3]:
annotations_dir = '../datasets/2021_annotations/'
cols2drop = ['View', 'Channel', 'Low Freq (Hz)', 'High Freq (Hz)', 'Delta Time (s)', 'Delta Freq (Hz)',
       'Avg Power Density (dB FS/Hz)']

# load annotations

In [4]:
df_list = []
for filename in os.listdir(annotations_dir):
    try:
        #         print(filename)
        annotation_file_path = os.path.join(annotations_dir, filename)
        small_df = load_n_adapt_raven_annotation_table_to_dv_dataset_requirements(annotation_file_path)
        df_list.append(small_df)
    except UnicodeDecodeError:
        print('exception:', 'UnicodeDecodeError')
        print('occured in file:', filename)
        continue
    except Exception as e:
        print('exception:', e)
        print('occured in file:', filename)
        continue
        

print(len(df_list))
df_all_annotations = pd.concat(df_list)
df_all_annotations = df_all_annotations.drop(cols2drop, axis=1)
# df_all_annotations.head()

exception: UnicodeDecodeError
occured in file: 210904-093942_Tr2.Table.1.selections.txt
exception: UnicodeDecodeError
occured in file: 210904-093942_Tr1.Table.1.selections- Annotated.txt
exception: 'end_time'
occured in file: 210904-111316_Tr2restofrecord.txt
33


In [5]:
df_all_annotations.shape
df_all_annotations.head()
df_all_annotations.tail()

(14898, 6)

,Selection,begin_time,end_time,Annotation,filename,call_length
0,1,2.008108,2.699544,NaN,210825-135601_Tr1,0.691436
1,3,5.619289,6.185679,NaN,210825-135601_Tr1,0.566389
2,4,8.171719,8.716042,NaN,210825-135601_Tr1,0.544322
3,5,11.245463,12.098725,NaN,210825-135601_Tr1,0.853262
4,6,16.927039,17.456650,NaN,210825-135601_Tr1,0.529611


,Selection,begin_time,end_time,Annotation,filename,call_length
763,777,1193.463021,1194.461811,NaN,210824-104507_Tr1.txt,0.998790
764,778,1197.903048,1198.732551,NaN,210824-104507_Tr1.txt,0.829504
765,779,1209.800186,1211.069834,NaN,210824-104507_Tr1.txt,1.269648
766,780,1213.084343,1214.328598,NaN,210824-104507_Tr1.txt,1.244255
767,781,1214.514813,1215.005744,NaN,210824-104507_Tr1.txt,0.490931


# explore labels

In [6]:
df_all_annotations.Annotation.unique()

array([nan, '?', 'Dolphins?', 'un', 'sc', 'un ', 'un !!! What ?', 'sc ?',
       'SC', 'd', 'un- weird whale sound probably', 'un d/w', 'dolphin',
       'd / cs ?', 'sc \\ d \\ un', 'cs ?', 'un \\ d ?', 'sc ? un',
       'few calls in this annotation', 'baby whale?', 'song (s)', 's',
       'un/d', 'un ?', 'w & un', '!', 'sc?', 'd?', 'ד'], dtype=object)

In [7]:
df_all_annotations.Annotation.value_counts()

sc                                915
un                                449
d                                 320
s                                 142
?                                  47
sc ?                               14
un d/w                              7
d?                                  4
dolphin                             3
Dolphins?                           3
SC                                  2
un/d                                2
un ?                                1
!                                   1
w & un                              1
baby whale?                         1
sc?                                 1
song (s)                            1
cs ?                                1
few calls in this annotation        1
sc ? un                             1
un \ d ?                            1
sc \ d \ un                         1
d / cs ?                            1
un- weird whale sound probably      1
un !!! What ?                       1
un          

For labels that appear only once - find location

In [8]:
all_unique_labels = df_all_annotations.Annotation.value_counts()[
    df_all_annotations.Annotation.value_counts() == 1].index.values

In [9]:
all_unique_labels

array(['un ?', '!', 'w & un', 'baby whale?', 'sc?', 'song (s)', 'cs ?',
       'few calls in this annotation', 'sc ? un', 'un \\ d ?',
       'sc \\ d \\ un', 'd / cs ?', 'un- weird whale sound probably',
       'un !!! What ?', 'un ', 'ד'], dtype=object)

In [10]:
df_unique_labels = df_all_annotations[df_all_annotations.Annotation.isin(all_unique_labels)].copy()

In [11]:
df_unique_labels.columns

Index(['Selection', 'begin_time', 'end_time', 'Annotation', 'filename',
       'call_length'],
      dtype='object')

In [14]:
cols2keep = ['Selection', 'begin_time', 'end_time', 'Annotation', 'filename']

In [15]:
df_unique_labels = df_unique_labels[cols2keep]

In [16]:
df_unique_labels.to_csv('unique_labels_info.csv', index=False)

In [17]:
def change_annotations(df_of_annotations: pd.DataFrame, annotations_to_change: list, target_value: str) -> None:
    """change specified annotations to target value. do this inplace."""
    df_of_annotations.replace(to_replace=annotations_to_change, value=target_value, inplace=True, limit=None, regex=False)
    return

def filter_annotations(df_of_annotations: pd.DataFrame, annotations_to_keep: list) -> None:
    """turn all annotations that were not specified as "keep", into 'bg'. do this inplace."""
    all_annotations = df_of_annotations.Annotation.unique()
    annotations_to_filter = list(set(all_annotations).difference(annotations_to_keep))
    df_of_annotations.replace(to_replace=annotations_to_filter, value='bg', inplace=True, limit=None, regex=False)
    return

In [18]:
annotations_to_change = ['SC', 'sc ?', 'un- weird whale sound probably', 'cs ?', 'baby whale?', 'song (s)', 's', 'sc?']
annotations_to_keep = ['sc']
target_value = 'sc'

change_annotations(df_all_annotations, annotations_to_change, target_value)
filter_annotations(df_all_annotations, annotations_to_keep)

In [19]:
df_all_annotations.Annotation.unique()
df_all_annotations.shape

array(['bg', 'sc'], dtype=object)

(14898, 6)

# label background

Methodology: all gaps between annotated segments are considered background

In [20]:
df_all_annotations.shape
df_all_annotations.filename.nunique()
df_all_annotations.head()

(14898, 6)

33

,Selection,begin_time,end_time,Annotation,filename,call_length
0,1,2.008108,2.699544,bg,210825-135601_Tr1,0.691436
1,3,5.619289,6.185679,bg,210825-135601_Tr1,0.566389
2,4,8.171719,8.716042,bg,210825-135601_Tr1,0.544322
3,5,11.245463,12.098725,bg,210825-135601_Tr1,0.853262
4,6,16.927039,17.456650,bg,210825-135601_Tr1,0.529611


In [21]:
df = df_all_annotations


In [83]:
# for filename in df.filename.unique():
#     print(filename)

## merge overlapping calls

In [22]:

def merge_overlapping_calls(df: pd.DataFrame) -> pd.DataFrame:
    """
    Receives an annotation dataframe with (possibly) overlapping calls, and goes through merge-and-drop iterations until
    no more overlaps are found.
    :param df: Pandas DataFrame with the following columns: ['filename', 'begin_time', 'end_time']
    :return: pd.DataFrame
    """
    df = df.sort_values(['filename', 'begin_time']).reset_index(drop=True)
    df = reset_overlap_accessory_columns(df)
    df = mark_overlapping_rows(df)

    while 1 in df.overlap.unique():
        df = merge_overlapping_rows(df)
        df = reset_overlap_accessory_columns(df)
        df = mark_overlapping_rows(df)

    df = df.drop(['overlap', 'next_begin_time', 'next_end_time'], axis=1)
    return df


def merge_overlapping_rows(df) -> pd.DataFrame:
    """
    Merge (and drop) overlapping rows.
    """
    df.loc[df.overlap == 1, 'end_time'] = df[df.overlap == 1]['next_end_time']
    df = df.drop_duplicates(subset=['filename', 'end_time'], keep='first')
    return df


def reset_overlap_accessory_columns(df) -> pd.DataFrame:
    df['overlap'] = np.NaN
    df['next_begin_time'] = df.groupby('filename').begin_time.shift(-1)
    df['next_end_time'] = df.groupby('filename').end_time.shift(-1)
    return df


def mark_overlapping_rows(df) -> pd.DataFrame:
    df.loc[df.next_begin_time < df.end_time, 'overlap'] = 1
    return df

# def mark_non_overlapping_rows(df) -> pd.DataFrame:
#     """
#     Merely for sanity check.
#     """
#     df.loc[(df.begin_time > df.end_time.shift(1)) & (df.end_time < df.begin_time.shift(-1)), 'overlap'] = 0
#     return df


In [23]:
df_no_overlap = merge_overlapping_calls(df)

In [24]:
df_no_overlap.shape

(11099, 6)

In [25]:
df.shape
df_no_overlap.shape

(14898, 6)

(11099, 6)

In [26]:
df.head()

,Selection,begin_time,end_time,Annotation,filename,call_length
0,1,2.008108,2.699544,bg,210825-135601_Tr1,0.691436
1,3,5.619289,6.185679,bg,210825-135601_Tr1,0.566389
2,4,8.171719,8.716042,bg,210825-135601_Tr1,0.544322
3,5,11.245463,12.098725,bg,210825-135601_Tr1,0.853262
4,6,16.927039,17.456650,bg,210825-135601_Tr1,0.529611


In [27]:
df_no_overlap.sort_values(['filename', 'begin_time']).head(20)

,Selection,begin_time,end_time,Annotation,filename,call_length
0,1,59.532117,60.937416,sc,180913_081527 (1),1.115795
2,3,62.137650,63.434385,sc,180913_081527 (1),1.471644
4,178,64.068321,65.576152,sc,180913_081527 (1),1.507831
5,5,66.978572,68.263244,sc,180913_081527 (1),1.284672
6,6,68.329589,69.511729,sc,180913_081527 (1),1.182140
7,7,71.271167,72.754873,sc,180913_081527 (1),1.887805
9,10,74.972095,76.172329,sc,180913_081527 (1),1.109764
11,12,77.059941,78.061142,sc,180913_081527 (1),1.151983
13,14,80.034604,81.650999,sc,180913_081527 (1),1.616395
14,15,82.201279,83.570390,sc,180913_081527 (1),1.369111


## mark gaps as background

In [28]:
df_bg = df_no_overlap[['filename', 'begin_time', 'end_time', 'call_length']].sort_values(['filename', 'begin_time']).reset_index(drop=True).copy()


In [29]:
df_bg.head()

,filename,begin_time,end_time
0,180913_081527 (1),59.532117,60.937416
1,180913_081527 (1),62.137650,63.434385
2,180913_081527 (1),64.068321,65.576152
3,180913_081527 (1),66.978572,68.263244
4,180913_081527 (1),68.329589,69.511729


In [30]:
df_bg['next_begin_time'] = df_bg.groupby('filename').begin_time.shift(-1)
# df_bg['next_end_time'] = df_bg.groupby('filename').end_time.shift(-1)

In [31]:

df_bg = df_bg.rename({'end_time': 'bg_begin_time', 'next_begin_time': 'bg_end_time'}, axis=1)

Manually look at shifts between different recordings to make sure there aren't mix-ups:

In [334]:
df_bg.filename.unique()
df_bg.filename.value_counts()

array(['180913_081527 (1)', '210824-095226_Tr1.txt',
       '210824-095226_Tr2.txt', '210824-100209_Tr1.txt',
       '210824-100209_Tr2.txt', '210824-104507_Tr1',
       '210824-104507_Tr1.txt', '210824-115331_Tr1.txt',
       '210824-125439_Tr1.txt', '210825-102141_Tr1.txt',
       '210825-112937_Tr1.txt', '210825-112937_Tr2.txt',
       '210825-132034_Tr1', '210825-132034_Tr2', '210825-135601_Tr1',
       '210825-135601_Tr1.Table.1.selections (1).txt',
       '210825-135601_Tr1.txt', '210826-083608_Tr1', '210827-081513_Tr1',
       '210827-081513_Tr2', '210827-090209_Tr1',
       '210827-133618_Tr1.Table.1.selections.donetxt.txt',
       '210827-133618_Tr2', '210828-080644_Tr1', '210903-095104_Tr1',
       '210903-095104_Tr2', '210903-110841_Tr1', '210903-110841_Tr2',
       '210904-074321_Tr1', '210904-111316_Tr2(first 4 minutes)(1).txt',
       '210904-111316_Tr2(first 4 minutes).txt', '210904-111316_Tr2.txt',
       '25-115438_Tr2'], dtype=object)

210824-115331_Tr1.txt                               736
210904-074321_Tr1                                   687
210824-104507_Tr1.txt                               648
210826-083608_Tr1                                   616
210824-125439_Tr1.txt                               596
210827-081513_Tr2                                   589
210828-080644_Tr1                                   538
210825-135601_Tr1.txt                               513
210825-135601_Tr1                                   479
210825-135601_Tr1.Table.1.selections (1).txt        479
210827-081513_Tr1                                   466
210825-102141_Tr1.txt                               444
210827-090209_Tr1                                   408
210904-111316_Tr2(first 4 minutes)(1).txt           380
210904-111316_Tr2(first 4 minutes).txt              380
210904-111316_Tr2.txt                               336
210825-132034_Tr2                                   310
210825-132034_Tr1                               

In [32]:
df_bg.loc[110:130]

,filename,begin_time,bg_begin_time,bg_end_time
110,180913_081527 (1),278.692123,280.705105,281.261778
111,180913_081527 (1),281.261778,282.387683,283.243070
112,180913_081527 (1),283.243070,284.550733,286.397010
113,180913_081527 (1),286.397010,287.420560,288.495288
114,180913_081527 (1),288.495288,289.297068,289.842962
115,180913_081527 (1),289.842962,290.542387,292.067182
116,180913_081527 (1),292.067182,292.800726,296.314915
117,180913_081527 (1),296.314915,298.208482,300.762484
118,180913_081527 (1),300.762484,301.683679,NaN
119,210824-095226_Tr1.txt,18.700366,19.764583,20.169273


In [33]:
df_bg.tail()

,filename,begin_time,bg_begin_time,bg_end_time
11094,25-115438_Tr2,579.313619,580.553433,580.662828
11095,25-115438_Tr2,580.662828,581.683851,581.838828
11096,25-115438_Tr2,581.838828,582.759572,583.115107
11097,25-115438_Tr2,583.115107,584.281991,584.327572
11098,25-115438_Tr2,584.327572,585.175386,NaN


In [34]:
df_bg[df_bg.filename=='25-115438_Tr2'].index.min()

10907

In [35]:
df_bg.loc[10900:10917]

,filename,begin_time,bg_begin_time,bg_end_time
10900,210904-111316_Tr2.txt,1112.462071,1113.903744,1113.952892
10901,210904-111316_Tr2.txt,1113.952892,1114.804789,1115.282497
10902,210904-111316_Tr2.txt,1115.282497,1115.683872,1116.437244
10903,210904-111316_Tr2.txt,1116.437244,1116.928723,1117.207228
10904,210904-111316_Tr2.txt,1117.207228,1118.345822,1118.909808
10905,210904-111316_Tr2.txt,1118.909808,1120.023828,1120.341310
10906,210904-111316_Tr2.txt,1120.341310,1120.800024,NaN
10907,25-115438_Tr2,14.210034,14.976796,17.372924
10908,25-115438_Tr2,17.372924,18.377055,21.126465
10909,25-115438_Tr2,21.126465,21.519430,23.514355


## Finalize bg annotation df

In [36]:
df_bg.filename.nunique()

33

In [37]:
df_bg.shape
df_bg = df_bg[~df_bg.bg_end_time.isna()]
df_bg.shape

(11099, 4)

(11066, 4)

In [38]:
df_bg = df_bg.drop('begin_time', axis=1)

In [39]:
df_bg['Annotation'] = 'bg'

In [43]:
df_bg

,filename,bg_begin_time,bg_end_time,Annotation
0,180913_081527 (1),60.937416,62.137650,bg
1,180913_081527 (1),63.434385,64.068321,bg
2,180913_081527 (1),65.576152,66.978572,bg
3,180913_081527 (1),68.263244,68.329589,bg
4,180913_081527 (1),69.511729,71.271167,bg
...,...,...,...,...
11093,25-115438_Tr2,577.526285,579.313619,bg
11094,25-115438_Tr2,580.553433,580.662828,bg
11095,25-115438_Tr2,581.683851,581.838828,bg
11096,25-115438_Tr2,582.759572,583.115107,bg


In [49]:
df_bg = df_bg.rename({'bg_begin_time': 'begin_time', 'bg_end_time': 'end_time'}, axis=1)

In [52]:
df_concat = pd.concat([df_bg, df_no_overlap[['begin_time', 'end_time', 'filename', 'call_length', 'Annotation']]])

In [53]:
df_concat = df_concat.rename({'Annotation': 'label'}, axis=1)

In [54]:
df_concat.head()

,filename,begin_time,end_time,label,call_length
0,180913_081527 (1),60.937416,62.137650,bg,NaN
1,180913_081527 (1),63.434385,64.068321,bg,NaN
2,180913_081527 (1),65.576152,66.978572,bg,NaN
3,180913_081527 (1),68.263244,68.329589,bg,NaN
4,180913_081527 (1),69.511729,71.271167,bg,NaN


In [56]:
df_concat['call_length'] = df_concat['end_time'] - df_concat['begin_time']

In [60]:
df_concat.label.value_counts()

bg    21340
sc      825
Name: label, dtype: int64

In [62]:
df_concat.to_csv('../datasets/mozambique_2021/2021_data_prepped_with_bg.csv', index = False)

In [1]:
def check_if_file_exists(self, annotation_file_path, extension):
    annotation_file_path.with_suffix(extension).exists()

In [2]:
pwd

'/home/rafael/DeepVoice/soundbay/notebooks'

In [1]:
path1 = '../tests/assets/mozambique2021/recordings'
path2 = '../datasets/2021_annotations'

In [2]:
import os

In [3]:
path1

'../tests/assets/mozambique2021/recordings'

In [4]:
for file in os.listdir(path1):
    print(file)

210903-121021_Tr2.wav
210825-135601_Tr1.wav
210831-132135_Tr1.wav
210824-125439_Tr2.wav
210827-124552_Tr2.wav
210825-132034_Tr1.wav
210824-100209_Tr2.wav
210903-110841_Tr1.wav
210826-110152_Tr2.wav
210824-115331_Tr2.wav
210824-104507_Tr1.wav
210831-132118_Tr1.wav
210825-132034_Tr2.wav
210824-100209_Tr1.wav
210831-075120_Tr1.wav
210831-103412_Tr2.wav
210901-100632_Tr2.wav
210824-095127_Tr2.wav
210901-100632_Tr1.wav
210903-080106_Tr1.wav
210825-102141_Tr1.wav
210903-080106_Tr2.wav
210901-113907_Tr2.wav
210904-093942_Tr1.wav
210831-082752_Tr1.wav
210904-111316_Tr1.wav
210825-112937_Tr2.wav
210901-083131_Tr2.wav
210825-112937_Tr1.wav
210826-101250_Tr1.wav
210828-094813_Tr1.wav
210902-133300_Tr2.wav
210824-104456_Tr1.wav
210828-094813_Tr2.wav
210904-080440_Tr2.wav
210827-133618_Tr1.wav
210825-135601_Tr2.wav
210827-110144_Tr2.wav
210901-073117_Tr2.wav
210902-160029_Tr2.wav
210902-121904_Tr2.wav
210826-083608_Tr1.wav
210831-122745_Tr1.wav
210826-110152_Tr1.wav
210827-090209_Tr1.wav
210824-095

In [5]:
import re
for file in os.listdir(path2):
    print(file)

210825-135601_Tr1.Table.1.selections.txt
210825-135601_Tr1.Table.1.selections (1).txt
210904-111316_Tr2(first 4 minutes)(1).txt
210827-133618_Tr2.Table.1.selections.txt
210904-093942_Tr2.Table.1.selections.txt
210828-080644_Tr1.Table.1.selections.txt
210904-111316_Tr2.txt
210827-081513_Tr1.Table.1.selections.txt
210825-132034_Tr1.Table.1.selections.txt
210824-104507_Tr1.Table.1.selections.txt
210825-112937_Tr1.txt
210904-074321_Tr1.Table.1.selections.txt
25-115438_Tr2.Table.1.selections.txt
210903-110841_Tr2.Table.1.selections.txt
210825-102141_Tr1.txt
210903-095104_Tr2.Table.1.selections.txt
210825-135601_Tr1.txt
210824-125439_Tr1.txt
180913_081527 (1).Table.1.selections.txt
210824-115331_Tr1.txt
210825-112937_Tr2.txt
210825-132034_Tr2.Table.1.selections.txt
210827-133618_Tr1.Table.1.selections.donetxt.txt
210826-083608_Tr1.Table.1.selections.txt
210827-081513_Tr2.Table.1.selections.txt
210824-100209_Tr1.txt
210824-095226_Tr2.txt
210904-111316_Tr2(first 4 minutes).txt
210827-090209_Tr

In [7]:
import filecmp
filecmp.cmp(f'{path2}/210904-111316_Tr2.txt', f'{path2}/210904-111316_Tr2(first 4 minutes).txt')


False

In [ ]:
annotation_filename_dict = {'210825-135601_Tr1.Table.1.selections.txt' : '210825-135601_Tr1.wav',
               '210825-135601_Tr1.Table.1.selections (1).txt': 'duplicate',
               '210904-111316_Tr2.txt': '210904-111316_Tr2.wav', 
               '210904-111316_Tr2restofrecord.txt': 'corrupt',
               '210904-111316_Tr2(first 4 minutes)(1).txt': 'duplicate', 
               '210904-111316_Tr2(first 4 minutes).txt': 'duplicate',
               '210827-133618_Tr2.Table.1.selections.txt': '210827-133618_Tr2.wav',
               '210904-093942_Tr2.Table.1.selections.txt': '210904-093942_Tr2.wav',
               '210828-080644_Tr1.Table.1.selections.txt': '210828-080644_Tr1.wav',
               '210827-081513_Tr1.Table.1.selections.txt': '210827-081513_Tr1.wav',
               '210825-132034_Tr1.Table.1.selections.txt': '210825-132034_Tr1.wav',
               '210824-104507_Tr1.Table.1.selections.txt': '210824-104507_Tr1.wav',
               '210824-104507_Tr1.txt': '210824-104507_Tr1.wav',
               '210825-112937_Tr1.txt' : '210825-112937_Tr1.wav',
               '210904-074321_Tr1.Table.1.selections.txt': '210904-074321_Tr1.wav',
               '25-115438_Tr2.Table.1.selections.txt': 'unknown',
               '210903-110841_Tr2.Table.1.selections.txt': '210903-110841_Tr2.wav',
               '210825-102141_Tr1.txt': '210825-102141_Tr1.wav',
               '210903-095104_Tr2.Table.1.selections.txt': '210903-095104_Tr2.wav',
               '210903-095104_Tr1.Table.1.selections.txt': '210903-095104_Tr1.wav',
               '210825-135601_Tr1.txt': '210825-135601_Tr1.wav',
               '210824-125439_Tr1.txt': '210824-125439_Tr1.wav',
               '180913_081527 (1).Table.1.selections.txt': 'unknown',
               '210824-115331_Tr1.txt': '210824-115331_Tr1.wav',
               '210825-112937_Tr2.txt': '210825-112937_Tr2.wav',
               '210825-132034_Tr2.Table.1.selections.txt': '210825-132034_Tr2.wav',
               '210827-133618_Tr1.Table.1.selections.donetxt.txt': '210827-133618_Tr1.wav',
               '210826-083608_Tr1.Table.1.selections.txt': '210826-083608_Tr1.wav',
               '210827-081513_Tr2.Table.1.selections.txt': '210827-081513_Tr2.wav',
               '210824-100209_Tr1.txt': '210824-100209_Tr1.wav',
               '210824-095226_Tr2.txt': '210824-095226_Tr2.wav',
               '210827-090209_Tr1.Table.1.selections.txt': '210827-090209_Tr2.wav',
               '210903-110841_Tr1.Table.1.selections.txt': '210903-110841_Tr1.wav',
               '210824-100209_Tr2.txt': '210824-100209_Tr2.wav',
               '210824-095226_Tr1.txt' : '210824-095226_Tr1.wav',
               '210904-093942_Tr1.Table.1.selections- Annotated.txt': '210904-093942_Tr1.wav',
               '210903-095104_Tr1.Table.1.selections.txt': '210903-095104_Tr1.wav'
